<a href="https://colab.research.google.com/github/geosefuy/sthciux-cashless/blob/master/%5BSTHCIUX%5D%20Needfinding%20Study%20Exploratory%20Data%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
url = 'https://raw.githubusercontent.com/geosefuy/sthciux-cashless/master/cashless.csv'
responses_df = pd.read_csv(url)
responses_df

,Timestamp,Are you willing to participate in a separate interview session for the research at hand?,Contact Number,Email Address,Sex,Current Level of Education,Type of Institution,Average Monthly Allowance (PHP),Do you have a credit / debit card?,How often do you use your credit / debit card for payment transactions per month?,"Do you have a payment application (e.g. GCash, PayMaya)?",How often do you use these applications for payment transactions per month?,"On average, how often do use cashless transactions systems in your payment transactions per month?",On what type of transactions do you usually use cashless transactions systems. Select all that apply. (Answer N/A if you always pay in cash),"On average, how often do you encounter establishments that have cashless transactions systems per month?","From what you know or have encountered, which of the following establishments implement cashless transaction systems?","On average, how would you rate current cashless transaction systems?","In your opinion, what are the possible advantages when more people use cashless systems?","In your opinion, what are the possible disadvantages when more people use cashless systems?",Which of the following establishments do you want to implement / maintain cashless transaction systems? (Select all that apply)
0,11/12/2020 14:22:32,Yes,09171054000,shannon_gail_ho@dlsu.edu.ph,Female,Undergraduate,Private School / University,"1,000 - 4,999",Yes,4 times or less,Yes,5 - 7 times,5 - 7 times,"Food and Groceries, Office/School Supplies, Ce...",8 - 10 times,"Supermarkets, Appliance Centers, Online Stores",8,More efficient and faster process,Not all are capable of using them and some are...,"Supermarkets, Convenience Stores, Payment Cent..."
1,11/12/2020 14:29:15,No,NaN,NaN,Male,Undergraduate,Private School / University,"Below 1,000",No,4 times or less,Yes,4 times or less,4 times or less,"Cellular Load, Games/Movies/Entertainment",4 times or less,"Convenience Stores, Payment Centers, Fast Food...",6,There's the convenience of having your money d...,There's both an upside and downside to securit...,"Supermarkets, Convenience Stores, Sari-sari St..."
2,11/12/2020 14:37:57,Yes,09052803070,gab_delacruz@dlsu.edu.ph,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,4 times or less,Yes,8 - 10 times,5 - 7 times,"Food and Groceries, Office/School Supplies, Ce...",5 - 7 times,"Supermarkets, Department Stores, Appliance Cen...",9,"Less human contact, especially during the pand...","Prone to it being hacked, delay in arrival of ...","Supermarkets, Department Stores, Appliance Cen..."
3,11/12/2020 14:44:33,No,NaN,NaN,Female,Undergraduate,Private School / University,"8,000 - 11,999",Yes,4 times or less,Yes,11 - 14 times,11 - 14 times,"Cellular Load, Games/Movies/Entertainment, ele...",8 - 10 times,"Supermarkets, Convenience Stores, Department S...",6,"Society can progress, and more attention can b...",People would have to adjust to the new norm. T...,"Supermarkets, Convenience Stores, Sari-sari St..."
4,11/12/2020 14:51:18,Yes,09177199289,richard_zapanta@dlsu.edu.ph,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,4 times or less,No,NaN,4 times or less,"Food and Groceries, Appliances",5 - 7 times,"Supermarkets, Department Stores, Appliance Cen...",6,It is less hassle to bring cash and it would a...,It would be hard for those who don’t have any ...,"Supermarkets, Convenience Stores, Department S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,11/23/2020 22:04:24,Yes,09175278590,antonellosantos8@gmail.com,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,4 times or less,Yes,4 times or less,15 times or more,"Food and Groceries, Office/School Supplies, Ga...",8 - 10 times,"Supermarkets, Convenience Stores, Department S...",8,"Given the current pandemic, it lessens the tra...",The time it takes to verify the transactions m...,"Supermarkets, Convenience Stores, Sari-sari St...

#Drop unneeded data

In [4]:
responses_df = responses_df.drop(['Timestamp', 'Are you willing to participate in a separate interview session for the research at hand?', 'Contact Number', 'Email Address'], axis=1)

#Rename questions to shorter column names

In [5]:
responses_df.rename(columns = {'Average Monthly Allowance (PHP)': 'allowance', 'Do you have a credit / debit card?': 'card_use', 'How often do you use your credit / debit card for payment transactions per month?': 'card_usage'}, inplace=True)
responses_df.rename(columns = {'Do you have a payment application (e.g. GCash, PayMaya)?': 'pay_app_use', 'How often do you use these applications for payment transactions per month?': 'pay_app_usage', 'On average, how often do use cashless transactions systems in your payment transactions per month?': 'cashless_usage'}, inplace=True)
responses_df.rename(columns = {'On what type of transactions do you usually use cashless transactions systems. Select all that apply. (Answer N/A if you always pay in cash)':'cashless_use_type', 'On average, how often do you encounter establishments that have cashless transactions systems per month?': 'cashless_encounter'}, inplace=True)
responses_df.rename(columns = {'From what you know or have encountered, which of the following establishments implement cashless transaction systems?': 'cashless_implement', 'On average, how would you rate current cashless transaction systems?': 'rating', 'Which of the following establishments do you want to implement / maintain cashless transaction systems? (Select all that apply)': 'cashless_implement_want'}, inplace=True)
responses_df.rename(columns = {'In your opinion, what are the possible advantages when more people use cashless systems?': 'advantages', 'In your opinion, what are the possible disadvantages when more people use cashless systems?': 'disadvantages'}, inplace=True)
responses_df

,Sex,Current Level of Education,Type of Institution,allowance,card_use,card_usage,pay_app_use,pay_app_usage,cashless_usage,cashless_use_type,cashless_encounter,cashless_implement,rating,advantages,disadvantages,cashless_implement_want
0,Female,Undergraduate,Private School / University,"1,000 - 4,999",Yes,4 times or less,Yes,5 - 7 times,5 - 7 times,"Food and Groceries, Office/School Supplies, Ce...",8 - 10 times,"Supermarkets, Appliance Centers, Online Stores",8,More efficient and faster process,Not all are capable of using them and some are...,"Supermarkets, Convenience Stores, Payment Cent..."
1,Male,Undergraduate,Private School / University,"Below 1,000",No,4 times or less,Yes,4 times or less,4 times or less,"Cellular Load, Games/Movies/Entertainment",4 times or less,"Convenience Stores, Payment Centers, Fast Food...",6,There's the convenience of having your money d...,There's both an upside and downside to securit...,"Supermarkets, Convenience Stores, Sari-sari St..."
2,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,4 times or less,Yes,8 - 10 times,5 - 7 times,"Food and Groceries, Office/School Supplies, Ce...",5 - 7 times,"Supermarkets, Department Stores, Appliance Cen...",9,"Less human contact, especially during the pand...","Prone to it being hacked, delay in arrival of ...","Supermarkets, Department Stores, Appliance Cen..."
3,Female,Undergraduate,Private School / University,"8,000 - 11,999",Yes,4 times or less,Yes,11 - 14 times,11 - 14 times,"Cellular Load, Games/Movies/Entertainment, ele...",8 - 10 times,"Supermarkets, Convenience Stores, Department S...",6,"Society can progress, and more attention can b...",People would have to adjust to the new norm. T...,"Supermarkets, Convenience Stores, Sari-sari St..."
4,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,4 times or less,No,NaN,4 times or less,"Food and Groceries, Appliances",5 - 7 times,"Supermarkets, Department Stores, Appliance Cen...",6,It is less hassle to bring cash and it would a...,It would be hard for those who don’t have any ...,"Supermarkets, Convenience Stores, Department S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,4 times or less,Yes,4 times or less,15 times or more,"Food and Groceries, Office/School Supplies, Ga...",8 - 10 times,"Supermarkets, Convenience Stores, Department S...",8,"Given the current pandemic, it lessens the tra...",The time it takes to verify the transactions m...,"Supermarkets, Convenience Stores, Sari-sari St..."
64,Male,Undergraduate,Private School / University,"5,000 - 7,999",Yes,4 times or less,Yes,5 - 7 times,4 times or less,"N/A (I always pay in cash), Food and Groceries",5 - 7 times,"Supermarkets, Convenience Stores, Sari-sari st...",6,"Given the pandemic, it would help have less co...",Sometimes it is dependent on the net and the a...,"Supermarkets, Convenience Stores, Sari-sari St..."
65,Female,Undergraduate,Private School / University,"Below 1,000",Yes,4 times or less,Yes,4 times or less,4 times or less,Tuition Fees,4 times or less,"Supermarkets, Department Stores, Payment Cente...",6,"Convenient and since it's the pandemic period,...",It's hard for those who don't have mobile phon...,"Supermarkets, Convenience Stores, Payment Cent..."
66,Female,Undergraduate,Private School / University,"Below 1,000",Yes,4 times or less,Yes,4 times or less,4 times or less,"Food and Groceries, Office/School Supplies, On...",5 - 7 times,"Supermarkets, Department Stores, Appliance Cen...",8,More hygienic and efficient (no need to waste ...,Dependency on the more known companies may cre...,"Supermarkets, Department Stores, Appliance Cen..."


#Replace binned data removing redundant strings

In [7]:
responses_df['card_usage'] = responses_df['card_usage'].str.replace('4 times or less', '1 - 4')
responses_df['pay_app_usage'] = responses_df['pay_app_usage'].str.replace('4 times or less', '1 - 4')
responses_df['cashless_usage'] = responses_df['cashless_usage'].str.replace('4 times or less', '1 - 4')
responses_df['cashless_encounter'] = responses_df['cashless_encounter'].str.replace('4 times or less', '1 - 4')
responses_df['card_usage'] = responses_df['card_usage'].str.replace('15 times or more', '> 15')
responses_df['pay_app_usage'] = responses_df['pay_app_usage'].str.replace('15 times or more', '> 15')
responses_df['cashless_usage'] = responses_df['cashless_usage'].str.replace('15 times or more', '> 15')
responses_df['cashless_encounter'] = responses_df['cashless_encounter'].str.replace('15 times or more', '> 15')
responses_df['card_usage'] = responses_df['card_usage'].str.replace(' times', '')
responses_df['pay_app_usage'] = responses_df['pay_app_usage'].str.replace(' times', '')
responses_df['cashless_usage'] = responses_df['cashless_usage'].str.replace(' times', '')
responses_df['cashless_encounter'] = responses_df['cashless_encounter'].str.replace(' times', '')
opinions_df = responses_df
responses_df

,Sex,Current Level of Education,Type of Institution,allowance,card_use,card_usage,pay_app_use,pay_app_usage,cashless_usage,cashless_use_type,cashless_encounter,cashless_implement,rating,advantages,disadvantages,cashless_implement_want
0,Female,Undergraduate,Private School / University,"1,000 - 4,999",Yes,1 - 4,Yes,5 - 7,5 - 7,"Food and Groceries, Office/School Supplies, Ce...",8 - 10,"Supermarkets, Appliance Centers, Online Stores",8,More efficient and faster process,Not all are capable of using them and some are...,"Supermarkets, Convenience Stores, Payment Cent..."
1,Male,Undergraduate,Private School / University,"Below 1,000",No,1 - 4,Yes,1 - 4,1 - 4,"Cellular Load, Games/Movies/Entertainment",1 - 4,"Convenience Stores, Payment Centers, Fast Food...",6,There's the convenience of having your money d...,There's both an upside and downside to securit...,"Supermarkets, Convenience Stores, Sari-sari St..."
2,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,1 - 4,Yes,8 - 10,5 - 7,"Food and Groceries, Office/School Supplies, Ce...",5 - 7,"Supermarkets, Department Stores, Appliance Cen...",9,"Less human contact, especially during the pand...","Prone to it being hacked, delay in arrival of ...","Supermarkets, Department Stores, Appliance Cen..."
3,Female,Undergraduate,Private School / University,"8,000 - 11,999",Yes,1 - 4,Yes,11 - 14,11 - 14,"Cellular Load, Games/Movies/Entertainment, ele...",8 - 10,"Supermarkets, Convenience Stores, Department S...",6,"Society can progress, and more attention can b...",People would have to adjust to the new norm. T...,"Supermarkets, Convenience Stores, Sari-sari St..."
4,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,1 - 4,No,NaN,1 - 4,"Food and Groceries, Appliances",5 - 7,"Supermarkets, Department Stores, Appliance Cen...",6,It is less hassle to bring cash and it would a...,It would be hard for those who don’t have any ...,"Supermarkets, Convenience Stores, Department S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Male,Undergraduate,Private School / University,"1,000 - 4,999",Yes,1 - 4,Yes,1 - 4,> 15,"Food and Groceries, Office/School Supplies, Ga...",8 - 10,"Supermarkets, Convenience Stores, Department S...",8,"Given the current pandemic, it lessens the tra...",The time it takes to verify the transactions m...,"Supermarkets, Convenience Stores, Sari-sari St..."
64,Male,Undergraduate,Private School / University,"5,000 - 7,999",Yes,1 - 4,Yes,5 - 7,1 - 4,"N/A (I always pay in cash), Food and Groceries",5 - 7,"Supermarkets, Convenience Stores, Sari-sari st...",6,"Given the pandemic, it would help have less co...",Sometimes it is dependent on the net and the a...,"Supermarkets, Convenience Stores, Sari-sari St..."
65,Female,Undergraduate,Private School / University,"Below 1,000",Yes,1 - 4,Yes,1 - 4,1 - 4,Tuition Fees,1 - 4,"Supermarkets, Department Stores, Payment Cente...",6,"Convenient and since it's the pandemic period,...",It's hard for those who don't have mobile phon...,"Supermarkets, Convenience Stores, Payment Cent..."
66,Female,Undergraduate,Private School / University,"Below 1,000",Yes,1 - 4,Yes,1 - 4,1 - 4,"Food and Groceries, Office/School Supplies, On...",5 - 7,"Supermarkets, Department Stores, Appliance Cen...",8,More hygienic and efficient (no need to waste ...,Dependency on the more known companies may cre...,"Supermarkets, Department Stores, Appliance Cen..."


# One-hot Encoding on `cashless_use_type`, `cashless_implement`, and `cashless_implement_want`

In [8]:
cleaned = responses_df.cashless_use_type.str.split(', ', expand=True).stack()
cleaned

0   0        Food and Groceries
    1    Office/School Supplies
    2             Cellular Load
    3                  Clothing
1   0             Cellular Load
                  ...          
66  1    Office/School Supplies
    2           Online Shopping
67  0        Food and Groceries
    1             Cellular Load
    2                  Clothing
Length: 158, dtype: object

In [9]:
one_hot_use_type = pd.get_dummies(cleaned, prefix='use_type').groupby(level=0).sum()
one_hot_use_type

,use_type_Anything that’s purchase online,use_type_Appliances,use_type_Books,use_type_Cellular Load,use_type_Clothing,use_type_Delivery / Receiving payments,use_type_Donation,use_type_Food and Groceries,use_type_Games/Movies/Entertainment,use_type_Money transfer,use_type_N/A (I always pay in cash),use_type_Office/School Supplies,use_type_Online Shopping,use_type_Online Subscriptions,use_type_Recreational Equipment,use_type_Tech stuff: mouse,use_type_Tuition Fee,use_type_Tuition Fees,use_type_electricity,use_type_keyboards,use_type_lazada,use_type_pen tabs
0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
64,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
66,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0


In [10]:
cleaned = responses_df.cashless_implement.str.split(', ', expand=True).stack()
one_hot_implement = pd.get_dummies(cleaned, prefix='implement').groupby(level=0).sum()
cleaned = responses_df.cashless_implement_want.str.split(', ', expand=True).stack()
one_hot_want = pd.get_dummies(cleaned, prefix='implement_want').groupby(level=0).sum()

In [11]:
one_hot_use_type

,use_type_Anything that’s purchase online,use_type_Appliances,use_type_Books,use_type_Cellular Load,use_type_Clothing,use_type_Delivery / Receiving payments,use_type_Donation,use_type_Food and Groceries,use_type_Games/Movies/Entertainment,use_type_Money transfer,use_type_N/A (I always pay in cash),use_type_Office/School Supplies,use_type_Online Shopping,use_type_Online Subscriptions,use_type_Recreational Equipment,use_type_Tech stuff: mouse,use_type_Tuition Fee,use_type_Tuition Fees,use_type_electricity,use_type_keyboards,use_type_lazada,use_type_pen tabs
0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
64,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
66,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0


In [12]:
one_hot_implement

,implement_Appliance Centers,implement_Convenience Stores,implement_Department Stores,implement_Fast Food Chains,implement_Food delivery services,implement_Jeepneys,implement_Online Stores,implement_Payment Centers,implement_Private Transportation (Taxis,implement_Public Transportation (Buses,implement_Rail Systems),implement_Restaurants,implement_Sari-sari stores,implement_Supermarkets,implement_TNVS)
0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0
2,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0
3,0,1,1,1,1,0,1,0,0,0,0,0,0,1,0
4,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,1,1,1,1,1,0,1,1,0,0,0,1,0,1,0
64,0,1,1,1,1,0,1,1,0,0,0,1,1,1,0
65,0,0,1,0,1,1,1,1,0,1,1,0,0,1,0
66,1,0,1,1,1,0,1,1,0,0,0,1,0,1,0


In [13]:
one_hot_want

,implement_want_Appliance Centers,implement_want_Convenience Stores,implement_want_Department Stores,implement_want_Donation drives,implement_want_Fast Food Chains,implement_want_Food Delivery Services,implement_want_Jeepneys,implement_want_Online Stores,implement_want_Payment Centers,implement_want_Pharmacies,implement_want_Private Transportation (Taxis,implement_want_Public Transportation (Buses,implement_want_Rail Systems),implement_want_Restaurants,implement_want_Sari-sari Stores,implement_want_Schools,implement_want_Supermarkets,implement_want_TNVS),implement_want_most have cashless maybe sari sari but i think it's impossible
0,0,1,0,0,0,1,1,1,1,0,1,1,1,0,0,0,1,1,0
1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0
2,1,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0
3,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0
4,1,1,1,0,1,1,1,1,0,0,1,1,1,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0
64,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0
65,0,1,0,0,0,0,1,1,1,0,1,1,1,0,0,0,1,1,0
66,1,0,1,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0


# Exploratory Data Analysis

## Dividing Students into Financial Brackets
When looking at the survey's overview, the proponents wanted to identify the habits of the students in terms of using cashless systems and its methods. However, since there can be various types of students with different financial standings, dividing them and finding habits for those specific groups will be done instead.

In [14]:
responses_df['allowance'].unique()

array(['1,000 - 4,999', 'Below 1,000', '8,000 - 11,999', '5,000 - 7,999',
       '15,000 - 19,999', '12,000 - 14,999', '25,000 and above'],
      dtype=object)

In [19]:
_1000 = responses_df[responses_df['allowance'] == 'Below 1,000']
_1000_4999 = responses_df[responses_df['allowance'] == '1,000 - 4,999']
_5000_7999 = responses_df[responses_df['allowance'] == '5,000 - 7,999']
_8000_11999 = responses_df[responses_df['allowance'] == '8,000 - 11,999']
_12000_14999 = responses_df[responses_df['allowance'] == '12,000 - 14,999']
_15000_19999 = responses_df[responses_df['allowance'] == '15,000 - 19,999']
_25000 = responses_df[responses_df['allowance'] == '25,000 and above']

,Sex,Current Level of Education,Type of Institution,allowance,card_use,card_usage,pay_app_use,pay_app_usage,cashless_usage,cashless_use_type,cashless_encounter,cashless_implement,rating,advantages,disadvantages,cashless_implement_want
43,Male,Undergraduate,Private School / University,"25,000 and above",No,NaN,Yes,1 - 4,1 - 4,"Office/School Supplies, Cellular Load, Games/M...",1 - 4,"Supermarkets, Convenience Stores, Department S...",6,It would be more efficient for the people and ...,People who are not aware or in the bottom clas...,"Supermarkets, Convenience Stores, Department S..."


# Opinions Analysis

In [ ]:
adv = opinions_df.iloc[:,-3:-2]
disadv = opinions_df.iloc[:,-2:-1]

In [ ]:
adv = adv.iloc[:, 0].str.replace(',', ' ')
adv = adv.str.replace('.', ' ')
disadv = disadv.iloc[:, 0].str.replace(',', ' ')
disadv = disadv.str.replace('.', ' ')

In [ ]:
adv

In [ ]:
disadv

In [ ]:
Counter(adv.str.split().sum()).most_common()

In [ ]:
Counter(disadv.str.split().sum()).most_common()